# CLUSTERS POLYGONS METHODS ENDPOINTS 

In [2]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


#### Import modules

In [242]:
# ---
# import modules

from warnings import filterwarnings as fws; fws('ignore')
import requests, json, pandas as pd, numpy as np, urllib
from matplotlib.path import Path as mpl_path
from datetime import datetime
from copy import deepcopy

# ---
# custom modules

from modules.mongo_requests import last_prediction_record
from modules.comando import comando_open_events #, polygons_geojson
from modules.waze import get_waze_partner_alerts
from modules.geojson_conversion import polygon_geojson

#### Custom functions

In [299]:
# ...

def filter_df(df, filter_dict={}):
    for key, value in filter_dict.items():
        if value is not list: value = [value]
        df = df[df.astype(str)[key].isin(value)]
    return df

#### Custom Polygon Class

In [318]:
class Polygon:

    def __init__(self, polygons, poly_id, urls=None, join_ids=None):
        if type(polygons) is pd.DataFrame:
            self.polygons_df = polygons
            polygons = polygon_geojson(polygons)
        else:
            self.polygons_df = None
        self.polygons = polygons
        self.poly_id = poly_id
        self.poly_ids = [feature['properties'][poly_id] for feature in polygons['features']]
        self.urls = urls
        self.join_ids = join_ids
        if urls is not None and type(join_ids) is str:
            self.join_ids = [join_ids] * len(urls)
            
    ## Get and append current monitoring data to json
    def polygons_status(self, base_id=None):
        if base_id is None: base_id = self.poly_id
        dfs = [pd.DataFrame(requests.get(url).json()).set_index(_id) for url, _id in zip(self.urls, self.join_ids)]
        return self.polygons_df.set_index(self.poly_id).join(dfs).reset_index().rename(columns={self.poly_id: base_id}).fillna('')

    # Returns polygon id list given a points list and a polygons geojson object
    
    def points_belong(self, df, point_id, coord_cols):
        if not len(df): return pd.Series()
        # Get events points array
        points = np.array(list(map(tuple, df[coord_cols].values)))
        # Get clusters polygons events dict
        events_poly = {}
        for feature in self.polygons['features']:
            cluster_id = feature['properties'][self.poly_id]
            poly = feature['geometry']['coordinates'][0]
            mpl_poly =  mpl_path(poly)
            points_msk = mpl_poly.contains_points(points)
            poly_events_df = df[points_msk]
            poly_events_ids = list(poly_events_df[point_id])
            for event_id in poly_events_ids:
                events_poly[event_id] = cluster_id
        # Update events dataframe with events polygons ids
        belong_msk = pd.Series(- np.ones(len(df)), index=df.index)
        for event_id, cluster_id in events_poly.items():
            belong_msk[df[point_id]==event_id] = cluster_id
        # Return extended open events
        return belong_msk.fillna('')

    # Appends 'polygon has points' columns to polygons dataset

    def has_points(self, df, point_id, coord_cols, key_id=None, prefix=''):
        if key_id is None: key_id = self.poly_id
        has_df_cols = [key_id] + [prefix + col for col in ['count', 'status', 'ids']]
        points_poly_ids = self.points_belong(df, point_id, coord_cols)
        has_df = []
        for _id in self.poly_ids:
            id_msk = points_poly_ids == _id
            n_points = sum(id_msk)
            has_points = int(n_points > 0)
            points_ids = list(df[point_id][id_msk])
            has_df.append([_id, n_points, has_points, points_ids])
        return pd.DataFrame(has_df, columns=has_df_cols)

---

## Get data

#### Comando pops dataset

In [319]:
# pop id map
comando_root = 'https://api.dados.rio/v2/adm_cor_comando'
pops_url = comando_root + '/pops'
pops_df = pd.DataFrame(requests.get(pops_url).json()['objeto'])
pops_map = pops_df.set_index('id')['titulo'].to_dict()

#### Polygons dataset

In [320]:
#### Load clusters
clusters_df = pd.read_csv('static/clusters/clusters_micro.csv')
clusters_df = clusters_df[clusters_df['sublabel'] != -1]

#### Sample point dataset

In [321]:
comando = comando_open_events()
waze = get_waze_partner_alerts()

Comando open events: Request attempt (1) 


---

## Clusters endpoint settings

In [430]:
import urllib

#### Clusters geojson properties settings

def prefix_url(root, params, col_type, col_prefix='prefix'):
    params = [urllib.parse.urlencode({col_type: value, col_prefix: key}) for key, value in params.items()]
    return [root + '?'+ param for param in params]

# ---
# columns settings

geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']

clusters_drop = [
    'radius', 'horizontal_perimeter', 'density_circle', 'density_box',
    'vertical_perimeter', 'lng_center', 'title', 'lat_center'
]

prob_drop = ['_id', 'cluster', 'range']
comando_drop = []
waze_drop = []

drop_cols = clusters_drop + prob_drop + comando_drop + waze_drop

# ---
# urls settings

api_root = 'http://127.0.0.1:5000/'

comando_types = {
    'bolsão': 5,
    'alagamento_enchente': 6,
    'alagamento': 31,
    'lâmina': 33,
    'sirene': 30,
    'enchente': 32,
    'vazamento': 16,
}

comando_clusters_url = api_root + 'comando/clusters'
comando_urls = prefix_url(comando_clusters_url, params=comando_types, col_type='pop_id')

waze_types = {
    'waze_flood': 'HAZARD_WEATHER_FLOOD'
}

waze_clusters_url = api_root + 'waze/clusters'
waze_urls = prefix_url(waze_clusters_url, params=waze_types, col_type='type')

urls = [api_root + 'predict'] + comando_urls + waze_urls

urls, drop_cols

(['http://127.0.0.1:5000/predict',
  'http://127.0.0.1:5000/comando/clusters?pop_id=5&prefix=bols%C3%A3o',
  'http://127.0.0.1:5000/comando/clusters?pop_id=6&prefix=alagamento_enchente',
  'http://127.0.0.1:5000/comando/clusters?pop_id=31&prefix=alagamento',
  'http://127.0.0.1:5000/comando/clusters?pop_id=33&prefix=l%C3%A2mina',
  'http://127.0.0.1:5000/comando/clusters?pop_id=30&prefix=sirene',
  'http://127.0.0.1:5000/comando/clusters?pop_id=32&prefix=enchente',
  'http://127.0.0.1:5000/comando/clusters?pop_id=16&prefix=vazamento',
  'http://127.0.0.1:5000/waze/clusters?type=HAZARD_WEATHER_FLOOD&prefix=waze_flood'],
 ['radius',
  'horizontal_perimeter',
  'density_circle',
  'density_box',
  'vertical_perimeter',
  'lng_center',
  'title',
  'lat_center',
  '_id',
  'cluster',
  'range'])

### Get Polygon class instance

In [356]:
polygons = Polygon(clusters_df, poly_id='sublabel', urls=urls, join_ids='cluster_id')
# polygons = Polygon(clusters_geojson, poly_id='sublabel')

### Get comando points polygons ids

In [357]:
point_id = 'id'
key_id = 'cluster_id'
coord_cols = ['latitude', 'longitude']
prefix = 'comando_'

comando['cluster_id'] = polygons.points_belong(comando, point_id, coord_cols)

comando['cluster_id'].head()

0    -1.0
1    15.0
2    -1.0
3    -1.0
4    -1.0
Name: cluster_id, dtype: float64

### Get polygons points info

In [358]:
comando_status = polygons.has_points(comando, point_id, coord_cols, key_id=key_id, prefix=prefix)

comando_status.head()

,cluster_id,comando_count,comando_status,comando_ids
0,0,0,0,[]
1,1,1,1,[86887]
2,2,0,0,[]
3,3,0,0,[]
4,4,0,0,[]


### Requests dinamic clusters

In [359]:
clusters = polygons.polygons_status(base_id='cluster_id')

clusters.drop(drop_cols + geometry_cols, axis=1, inplace=True)

clusters.head()

,cluster_id,main_neighborhood,main_route,main_street_number_range,lat_centroid,lng_centroid,label_count,area_box,area_circle,timestamp,...,sirene_ids,enchente_count,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,waze_flood_count,waze_flood_status,waze_flood_ids
0,0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,-23.006631,-43.310232,114,0.047620,0.178174,2022-12-23 20:45:00.008000,...,[],0,0,[],0,0,[],0,0,[]
1,1,Catete,Rua do Catete,228 - 139,-22.926423,-43.176842,99,0.057771,0.077165,2022-12-23 20:45:00.008000,...,[],0,0,[],1,1,[86887],0,0,[]
2,2,Copacabana,Rua Tonelero,236 - 9,-22.966793,-43.185999,43,0.040122,0.042480,2022-12-23 20:45:00.008000,...,[],0,0,[],0,0,[],0,0,[]
3,3,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,-22.979735,-43.202540,36,0.039586,0.069447,2022-12-23 20:45:00.008000,...,[],0,0,[],0,0,[],0,0,[]
4,4,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,-23.012736,-43.298339,36,0.052712,0.167301,2022-12-23 20:45:00.008000,...,[],0,0,[],0,0,[],0,0,[]


### Dinamic clusters column fields

In [432]:
clusters.columns

Index(['cluster_id', 'main_neighborhood', 'main_route',
       'main_street_number_range', 'lat_centroid', 'lng_centroid',
       'label_count', 'area_box', 'area_circle', 'timestamp', 'date', 'time',
       'probability', 'confidence', 'label', 'bolsão_count', 'bolsão_status',
       'bolsão_ids', 'alagamento_enchente_count', 'alagamento_enchente_status',
       'alagamento_enchente_ids', 'alagamento_count', 'alagamento_status',
       'alagamento_ids', 'lâmina_count', 'lâmina_status', 'lâmina_ids',
       'sirene_count', 'sirene_status', 'sirene_ids', 'enchente_count',
       'enchente_status', 'enchente_ids', 'vazamento_count',
       'vazamento_status', 'vazamento_ids', 'waze_flood_count',
       'waze_flood_status', 'waze_flood_ids'],
      dtype='object')

## Add operatinal logic

In [427]:
status_names = {0: 'NORMALIDADE', 1: 'ATENÇÃO', 2: 'ALERTA', 3: 'PERÍGO'}

def get_clusters_status(drop_geometry=False):
    drop = drop_cols[:]
    if drop_geometry: drop += geometry_cols
    clusters = polygons.polygons_status(base_id=key_id)
    clusters.drop(drop, axis=1, inplace=True)
    return clusters

def get_logical_status(status):
    status_code = pd.Series(index=status.index)

    comando_alagamento = ['bolsão_status', 'alagamento_status', 'alagamento_enchente_status', 'enchente_status']
    alagamento = status[comando_alagamento].sum(1) > 0
    not_alagamento = alagamento.apply(lambda cond: not cond).astype(int)
    prob_or_lamina = (status[['label', 'lâmina_status']].sum(1) > 0).astype(int)
    alerta = pd.concat([not_alagamento, prob_or_lamina], 1).sum(1) == 2
    not_alerta = alerta.apply(lambda cond: not cond).astype(int)
    atencao = pd.concat([not_alagamento, not_alerta, status['waze_flood_status']], 1).sum(1) == 3
    not_atencao = atencao.apply(lambda cond: not cond).astype(int)
    normal = pd.concat([not_alagamento, not_alerta, not_atencao], 1).sum(1) == 3

    status_code[alagamento] = 3 # code 3
    print(status_code.isna().sum())
    status_code[alerta] = 2 # code 2
    print(status_code.isna().sum())
    status_code[atencao] = 1 # code 1
    print(status_code.isna().sum())
    status_code[normal] = 0 # code 0
    print(status_code.isna().sum())
    
    return status_code, status_code.map(status_names)

### Get status code and name

In [429]:
status = get_clusters_status(True)

status_code, status_name = get_logical_status(status)

status['status_code'] = status_code
status['status_name'] = status_name

status[['status_code', 'status_name']]

79
79
79
0


,status_code,status_name
0,0.0,NORMALIDADE
1,0.0,NORMALIDADE
2,0.0,NORMALIDADE
3,0.0,NORMALIDADE
4,0.0,NORMALIDADE
...,...,...
74,0.0,NORMALIDADE
75,0.0,NORMALIDADE
76,0.0,NORMALIDADE
77,0.0,NORMALIDADE
